<a href="https://colab.research.google.com/github/dharmendra098/debrid/blob/main/torrent_search_and_get_debrid_links_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
#1337x
!pip install 1337x
#piratesbay
!git clone https://github.com/gp2112/pyratebay.git
!pip install pyratebay/.

In [2]:
import pandas as pd
import requests
from py1337x import py1337x
import pyratebay
torrents = py1337x(proxy='1337x.to', cache='py1337xCache', cacheTime=500)
# Using 1337x.tw and saving the cache in sqlite database which expires after 500 seconds

In [3]:
api='debrid api here'

In [4]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Functions

In [5]:
df_torrents = pd.DataFrame({})
df_cached = pd.DataFrame({})
df_cloud = pd.DataFrame({})
df_files = pd.DataFrame({})
links = []
download_links = []

In [6]:
def file_list(dic):
    if isinstance(dic, dict):
        files_in_source = (
            pd.DataFrame([item for sublist in [list(d.items()) for d in dic['rd']] for item in sublist], columns=['sno', 'dic'])
            .drop_duplicates(subset='sno', keep='first')
            .reset_index(drop=True)
            
        )
        df = pd.concat([files_in_source.apply(lambda row: row.dic['filename'], axis = 1)
                        ,files_in_source.apply(lambda row: row.dic['filesize'], axis = 1)], axis=1)
        return df.to_numpy()
    else:
        return []

In [7]:
def size(n):
    n=n/1000000000
    gb= "{:.2f}".format(n)
    mb= "{:.2f}".format(n*1000)
    if(n>=1):
        return f"{gb}GB"
    else:
        return f"{mb}MB"    

In [8]:
def search_1337x(query, type_ ='All'):
    global df_torrents
    # get torrent list by search
    try:
        if(type_=='All'):
            results_dic = torrents.search(query)
        else:
            results_dic = torrents.search(query, category=type_)

        df = pd.DataFrame(results_dic['items'])
    except:
        print('site not accessible')
        return None
    
    
    # add info hash columns
    try:
        df['infoHash'] = 'NA'
        df['magnet'] = 'NA'
        for row in df.itertuples():
            try:
                torrent_info = torrents.info(link=row.link)
                df['infoHash'].iloc[row.Index] = torrent_info['infoHash']
                df['magnet'].iloc[row.Index] = torrent_info['magnetLink']
            except:
                pass
        df = df[df['infoHash'] != 'NA'] 

    except:
        print('info hash not added')
        return df[['name', 'seeders', 'leechers', 'size', 'time', 'uploader']]
    df_torrents = df[['name', 'seeders', 'leechers', 'size', 'time', 'uploader', 'infoHash', 'magnet']]

    return df_torrents[['name', 'seeders', 'leechers', 'size', 'time', 'uploader']]

In [9]:
def search_tpb(query):
    global df_torrents
    torrents = pyratebay.search(query)
    list_search_results=[]
    for torrent in torrents:
        list_search_results.append({'name':torrent.name, 'seeders': torrent.seeders, 'leechers': torrent.leechers, 'size':torrent.size, 'time': torrent.added, 'num_files': torrent.num_files, 'infoHash': torrent.info_hash, 'magnet': torrent.magnet()})

    df_torrents = pd.DataFrame(list_search_results)
    df_torrents['time'] = pd.to_datetime(df_torrents['time'],unit='s')
    df_torrents['size']=df_torrents['size'].astype(int).apply(size)

    return df_torrents[['name', 'seeders', 'leechers', 'size', 'time', 'num_files']]

In [10]:
def filter_cached():
        global df_cached
        df = df_torrents.copy()
        #check cached
        hash_list = '/'.join(df.loc[:,'infoHash'].to_list())
        url = f"https://api.real-debrid.com/rest/1.0/torrents/instantAvailability/{hash_list}"
        response = requests.get(
            url
            ,headers={"Authorization": f"Bearer {api}"}
        )
        df['cache_info'] = df.apply(lambda row: file_list(response.json()[row.infoHash.lower()]), axis = 1)
        df['number_of_files'] = df.apply(lambda row: len(row.cache_info), axis = 1)
        df=df[df['number_of_files']>0].reset_index(drop=True)
        df_cached=df

        return df_cached[['name', 'seeders', 'leechers', 'size', 'time', 'number_of_files']]

In [11]:
def get_debrid_link(i):
    global df_torrents, df_cloud, df_files, links
    # check link in cloud
    try:
        print('Cloud scanning')
        response = requests.get(
            'https://api.real-debrid.com/rest/1.0/torrents'
            ,headers={"Authorization": f"Bearer {api}"}
        )
        df_cloud = pd.DataFrame(response.json())
        info_hash = df_cached.loc[i, 'infoHash'].lower()
        matching_indexes = df_cloud.index[df_cloud['hash'] == info_hash].to_list()
        if len(matching_indexes):
            print('Found!')
            id_ = df_cloud.loc[matching_indexes[0], 'id']
            url = f"https://api.real-debrid.com/rest/1.0/torrents/info/{id_}"
            response = requests.get(
                url
                ,headers={"Authorization": f"Bearer {api}"}
            )
            links = response.json()['links']
            df_files= pd.DataFrame(response.json()['files']).drop(['id', 'selected'], axis =1).rename({'path':'filename'}, axis=1)
            df_files['size']=df_files['bytes'].apply(size)
            df_files.drop(['bytes'], axis=1, inplace=True)
            return df_files
        else:
            print('Pass')
    except:
        print('cloud checking failed')
        pass
            
            
    
    #add magnet to debrid
    try:
        magnet = df_cached.loc[i,'magnet']

        response = requests.post(
            'https://api.real-debrid.com/rest/1.0/torrents/addMagnet'
            ,{"magnet" : magnet}
            ,headers={"Authorization": f"Bearer {api}"}
        )
        if not response.ok:
            print("error adding magnet")
            return None
    except:
        print('error adding magnet')
        return None
    
    #start magnet link
    try:
        id_ = response.json()['id']
        url = f"https://api.real-debrid.com/rest/1.0/torrents/selectFiles/{id_}"
        response = requests.post(
            url
            ,{"files": "all"}
            ,headers={"Authorization": f"Bearer {api}"}
        )

        if not response.ok:
            print("error starting magnet")
            return None

    except:
        print('error starting magnet')
        return None


    #get cached magnet link
    try:
        url = f"https://api.real-debrid.com/rest/1.0/torrents/info/{id_}"
        response = requests.get(
            url
            ,headers={"Authorization": f"Bearer {api}"}
        )
        links = response.json()['links']
        if response.ok:
            print('magnet added')
        else:
            print('error getting files')
            return None
    except:
        print('error getting files')
        return None
    
    df_files= pd.DataFrame(response.json()['files']).drop(['id', 'selected'], axis =1).rename({'path':'filename'}, axis=1)
    df_files['size']=df_files['bytes'].apply(size)
    df_files.drop(['bytes'], axis=1, inplace=True)
    return df_files

In [12]:
def unrestrict(i=[-1]):
    global download_links
    if isinstance(i, int):
        i_=[]
        i_.append(i)
        i=i_
    
    result = []
    #get unrestricted link
    for j,link in enumerate(links):
        if j not in i and i!=[-1]:
            continue
        try:
            response = requests.post(
                'https://api.real-debrid.com/rest/1.0/unrestrict/link'
                ,{"link": link}
                ,headers={"Authorization": f"Bearer {api}"}
            )

                        
            if not response.ok:
                print('error unrestricting')
                return None

            response = response.json()
            result.append((response['filename'],response['download']))
            download_links.append(response['download'])


        except:
            print('error unrestricting')
            return None

    return result

# Run

In [13]:
search_tpb('gta v')
# search_1337x('gta v')
filter_cached()

,name,seeders,leechers,size,time,number_of_files
0,GTA V - Grand Theft Auto V - Rjaa,526,179,63.39GB,2019-08-20 10:38:02,11
1,GTA San Andreas-HOODLUM Plus SAMP Multiplayer,388,12,4.23GB,2012-06-21 07:41:21,4
2,[PC Game] Gta Vice City (Multi5) TNT Village,77,2,1.35GB,2015-05-05 22:58:39,1
3,GTA-IV v1.0.8.0 Repack Team-LiL,56,4,13.84GB,2020-03-05 14:37:23,9
4,Grand Theft Auto (GTA) San Andreas v1.06 Android,15,0,1.79GB,2014-10-27 00:46:00,3
5,Grand Theft Auto (GTA): San Andreas (PS2),11,0,2.79GB,2010-01-17 12:57:15,3
6,GTA Grand Theft Auto V (PS3),8,3,17.33GB,2016-04-22 20:27:55,4
7,PS2 - GTA San Andreas [1 DVD - PAL - MultiLang...,7,2,2.79GB,2010-04-27 00:20:50,29
8,PC Â» GTA IV PERFECT RIP directplay by globe@,4,0,4.76GB,2010-04-01 22:42:50,6
9,GTA Liberty City Stories [PSP],4,0,476.73MB,2013-12-16 04:15:36,1


In [14]:
get_debrid_link(0)

Cloud scanning
Found!


,filename,size
0,/GamesTorrents - Descargar Juegos Torrent Grat...,0.00MB
1,/Importante.url,0.00MB
2,/Instrucciones.url,0.00MB
3,/Crack/steam_api.ini,0.00MB
4,/reloaded.nfo,0.01MB
5,/Crack/socialclub.ini,0.01MB
6,/Crack/doge64.dll,0.12MB
7,/Crack/steam_api_ext64.dll,0.34MB
8,/Crack/socialclub.dll,0.68MB
9,/Crack/steam_api64.dll,1.61MB


In [15]:
unrestrict()

[('GTA V [wWw.GamesTorrents.TV].rar',
  'https://20.download.real-debrid.com/d/YHPIGDFBHVW5E/GTA%20V%20%5BwWw.GamesTorrents.TV%5D.rar')]

In [16]:
s=''
for link in download_links:
    s=s+link+'\n'
s

'https://20.download.real-debrid.com/d/YHPIGDFBHVW5E/GTA%20V%20%5BwWw.GamesTorrents.TV%5D.rar\n'